In [5]:
#1 create a dataset, put all samples in one file called BBCdata.txt

import os
#import string

BASE_DIR=r'databases\bbc dataset'
           
LABELS=['business','entertainment','politics','sport','tech']


with open('databases/BBCdata.txt', 'w', encoding='utf8') as outfile: 
        
    for label in LABELS:
            #dir='%s/%s' %(BASE_DIR, label)
        dir=os.path.join(BASE_DIR,label)
        for filename in os.listdir(dir):
            fullfilename='%s/%s' %(dir, filename)
            with open (fullfilename, 'rb') as file:
                text=file.read().decode(errors='replace').replace('\n',' ') 
                outfile.write('%s\t%s\t%s\n' %(label,filename,text)) 
   

In [6]:
#2  docs =[(class,txt),(class,txt),....] : convert data file to a list of tupels

docs=[]
with open ('databases/BBCdata.txt', 'r', encoding='utf8') as datafile:
    for row in datafile:
        parts=row.split('\t')
        doc=(parts[0], parts[2].strip())
        docs.append(doc)


In [7]:
#3 text preprocessing: text cleaning

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import porter


stemmer = porter.PorterStemmer() # stemming each word to the root

newtuple=[]

for doc in docs:   
        
    doc_label=doc[0]
    matn=doc[1]
    matn=matn.translate(str.maketrans('','',string.punctuation)) 
    matn=matn.lower()
    
    word_tokens=matn.split() 
    review = [stemmer.stem(word) for word in word_tokens if not word in set(stopwords.words('english'))]  
    
    review = " ".join(review) 
    
    newlist=(doc_label, review) 
    newtuple.append(newlist)
    

In [8]:
#4 classifier coniguration, and training

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split


xtrain, xtest, ytrain, ytest = train_test_split([i[1] for i in newtuple], [i[0] for i in newtuple], 
                                                    test_size=0.30, random_state=42, shuffle=True)
    
vectorizer=CountVectorizer(stop_words='english', ngram_range=[1,3],
                              min_df=3, analyzer='word')
    
dtm=vectorizer.fit_transform(xtrain)
NB=MultinomialNB().fit(dtm,ytrain)


In [12]:
#5 prediction and evaluation

def evaluate_classifier (title,classifier,vectorizer,xtest,ytest):
    xtest_tfidf=vectorizer.transform(xtest)
    y_pred=classifier.predict(xtest_tfidf)
    
    accuracy  = metrics.accuracy_score(ytest, y_pred)
    precision = metrics.precision_score(ytest,y_pred, average='micro') # default average='binary'
    recall    = metrics.recall_score(ytest,y_pred, average='micro')
    f1        = metrics.f1_score(ytest,y_pred, average='micro')
    
    
    print('%s accuracy: %f\t percision: %f\t recall: %f\t f1: %f\n' %(title,accuracy,precision,recall,f1))
    
evaluate_classifier('Nave Bayes  Train:\t', NB, vectorizer, xtrain, ytrain )
evaluate_classifier('Nave Bayes  Test:\t',  NB, vectorizer, xtest, ytest )

Nave Bayes  Train:	 accuracy: 0.997431	 percision: 0.997431	 recall: 0.997431	 f1: 0.997431

Nave Bayes  Test:	 accuracy: 0.976048	 percision: 0.976048	 recall: 0.976048	 f1: 0.976048



In [ ]:
#store the classifier and vectorizer:
import pickle

clf_filename='NB.pkl'
pickle.dump(NB, open(clf_filename,'wb')) #  (esme model train shode, open(esme file save shavanve)), shabakeye NB_model ro be esme clf_filename='NB.pkl' zakhire mikone

vec_filename='count_vectorizer.pkl'
pickle.dump(vectorizer, open(vec_filename,'wb'))